## Import

In [ ]:
!python3 -m pip install pyarrow

In [ ]:
import pandas as pd
import psycopg2 as pg
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grd
from matplotlib.ticker import PercentFormatter
from matplotlib import ticker
from textwrap import wrap
import time
import random
import math
from copy import deepcopy

import feather

from scipy.stats import binned_statistic
from scipy import stats
from sklearn.metrics import confusion_matrix

import seaborn as sns
import pycountry_convert as pc

import torch.nn as nn
import torch.nn.functional as F
import torch

from pandas.api.types import CategoricalDtype
from plotnine import *
from tqdm import tqdm

import os

In [ ]:
sns.set_theme()

In [ ]:
# NOTE: Best to NOT run this
import warnings
warnings.filterwarnings('ignore')

#### Import classes defined in separate files

In [ ]:
import sys; sys.path.append("/dhi_work/walter/warfarin/warfarin")
from models.smdp_dBCQ import discrete_BCQ, FC_Q
from utils.smdp_buffer import SMDPReplayBuffer
from helper_functions.model import Model
from helper_functions.graphs import Graphs
from helper_functions.threshold_model import ThresholdModel
from helper_functions.constants import Constants

## Load replay buffers

In [ ]:
# Directory locations
root_dir = "../"
save_folder = root_dir + "output/dBCQ" 
buffer_folder = root_dir + "data/replay_buffers"

In [ ]:
# Network parameters
num_actions = 7
state_dim = 56
suffix = "smdp"

In [ ]:
# Training parameters
lr = 5e-5
bcq_threshold = 0.3
iteration = 500000
hstates = 64
events_batch_size = 0
seed = 0

In [ ]:
buffer_name = f"actions_{num_actions}_state_{state_dim}_{suffix}"
savename = f"lr{lr}_bcq{bcq_threshold}_hstates{hstates}_evBatchsize{events_batch_size}_seed{seed}"

In [ ]:
folder_paths = {
    "results": f"{save_folder}/results/{buffer_name}/{savename}",
    "models": f"{save_folder}/models/{buffer_name}/{savename}",
    "buffers": f"{buffer_folder}/{buffer_name}"
}

In [ ]:
train_buffer = SMDPReplayBuffer(data_path=folder_paths["buffers"] + "/train_data", state_dim=state_dim)
val_buffer = SMDPReplayBuffer(data_path=folder_paths["buffers"] + "/val_data", state_dim=state_dim)

In [ ]:
train_buffer.load()
val_buffer.load()

In [ ]:
suffix = ""
baseline = pd.read_feather(f"../data/clean_data/baseline{suffix}.feather")

### Load model

In [ ]:
if iteration is not None:
    try:
        model_path = folder_paths["models"] + f"/checkpoint_{iteration}"
    except Exception as e:
        model_path = folder_paths["models"] + f"/checkpoint"
else:
    model_path = folder_paths["models"] + f"/checkpoint"

In [ ]:
# model_path = Model.get_model_path(suffix, folder="", lr=lr, iteration=iteration, threshold=bcq_threshold, hidden_states=hstates, events_batch_size=events_batch_size, seed=seed)
# model_path = folder_paths["models"]
model = Model(model_path, num_actions=num_actions, state_dim=state_dim, hidden_states=hstates, bcq_threshold=bcq_threshold)

### Load replay buffers

In [ ]:
ADV_EVENTS = ["STROKE", "HEM_STROKE", "MAJOR_BLEED"]

In [ ]:
train_buffer.data["TIME_TO_END"] = train_buffer.data.groupby('USUBJID_O_NEW').STUDY_WEEK.transform('max') - train_buffer.data.STUDY_WEEK
val_buffer.data["TIME_TO_END"] = val_buffer.data.groupby('USUBJID_O_NEW').STUDY_WEEK.transform('max') - val_buffer.data.STUDY_WEEK

nonsurvivor_ids = train_buffer.data[(train_buffer.data[ADV_EVENTS].sum(axis=1) > 0)]["USUBJID_O_NEW"].unique()
train_buffer.data["SURVIVOR_FLAG"] = np.where(train_buffer.data["USUBJID_O_NEW"].isin(nonsurvivor_ids), 0, 1)

nonsurvivor_ids = val_buffer.data[(val_buffer.data[ADV_EVENTS].sum(axis=1) > 0)]["USUBJID_O_NEW"].unique()
val_buffer.data["SURVIVOR_FLAG"] = np.where(val_buffer.data["USUBJID_O_NEW"].isin(nonsurvivor_ids), 0, 1)

train_buffer.data['INR_VALUE_ADJ'] = train_buffer.data['INR_VALUE'] * 4 + 0.5
train_buffer.data = SMDPReplayBuffer.get_ttr(train_buffer.data, colname='INR_VALUE_ADJ')
val_buffer.data['INR_VALUE_ADJ'] = val_buffer.data['INR_VALUE'] * 4 + 0.5
val_buffer.data = SMDPReplayBuffer.get_ttr(val_buffer.data, colname='INR_VALUE_ADJ')

train_buffer.data['TIME_BELOW_RANGE'] = train_buffer.data.groupby("USUBJID_O_NEW")["BELOW_RANGE"].cumsum() / (train_buffer.data.groupby("USUBJID_O_NEW")["BELOW_RANGE"].cumcount())
val_buffer.data['TIME_BELOW_RANGE'] = val_buffer.data.groupby("USUBJID_O_NEW")["BELOW_RANGE"].cumsum() / (val_buffer.data.groupby("USUBJID_O_NEW")["BELOW_RANGE"].cumcount())

train_buffer.data['TIME_ABOVE_RANGE'] = train_buffer.data.groupby("USUBJID_O_NEW")["ABOVE_RANGE"].cumsum() / (train_buffer.data.groupby("USUBJID_O_NEW")["ABOVE_RANGE"].cumcount())
val_buffer.data['TIME_ABOVE_RANGE'] = val_buffer.data.groupby("USUBJID_O_NEW")["ABOVE_RANGE"].cumsum() / (val_buffer.data.groupby("USUBJID_O_NEW")["ABOVE_RANGE"].cumcount())

train_buffer.data["INR_BIN"] = Model.bin_inr(train_buffer.data, colname="INR_VALUE_ADJ", num_bins=5)
val_buffer.data["INR_BIN"] = Model.bin_inr(val_buffer.data, colname="INR_VALUE_ADJ", num_bins=5)

In [ ]:
# Map back to original data to get the original patient IDs

data_dir = "../data/split_data/"
train_data = feather.read_dataframe(data_dir + "train_data.feather")
val_data = feather.read_dataframe(data_dir + "val_data.feather")

In [ ]:
id_mapping = pd.DataFrame(train_data.groupby("USUBJID_O_NEW")["SUBJID"].last()).reset_index()
train_buffer.data = train_buffer.data.merge(id_mapping, how="left", on="USUBJID_O_NEW")

In [ ]:
id_mapping = pd.DataFrame(val_data.groupby("USUBJID_O_NEW")["SUBJID"].last()).reset_index()
val_buffer.data = val_buffer.data.merge(id_mapping, how="left", on="USUBJID_O_NEW")

### Quick Stats!

In [ ]:
# Note: this state method is used to get the columns of the dataframe.
# I'm pretty sure we can get rid of the dependence on "state method" since we are already storing the dataframe but ok
state_method = 18

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))

sample_state1 = deepcopy(train_buffer.state)
model.get_model_results(sample_state1, state_method=state_method, num_actions=num_actions)
Graphs.plot_heatmap(model.df, axs=axes[0], num_actions=num_actions, return_norm2=False);
orig_title = axes[0].title.get_text()
axes[0].set_title(f"Training Data")

sample_state2 = deepcopy(val_buffer.state)
model.get_model_results(sample_state2, state_method=state_method, num_actions=num_actions)
Graphs.plot_heatmap(model.df, axs=axes[1], num_actions=num_actions, return_norm2=False);
axes[1].set_title(f"Validation Data")

fig.suptitle(orig_title, fontsize=16)
plt.tight_layout()

In [ ]:
incl_flags = True

In [ ]:
buffer = deepcopy(train_buffer)
sens, spec = Graphs.get_sens_wrapper(buffer, model, flipped=True, incl_flags=incl_flags)

buffer = deepcopy(val_buffer)
sens, spec = Graphs.get_sens_wrapper(buffer, model, flipped=True, incl_flags=incl_flags)

In [ ]:
buffer = deepcopy(train_buffer)
sens, spec = Graphs.get_sens_wrapper(buffer, model, flipped=True, only_direc=True, incl_flags=incl_flags)

buffer = deepcopy(val_buffer)
sens, spec = Graphs.get_sens_wrapper(buffer, model, flipped=True, only_direc=True, incl_flags=incl_flags)

In [ ]:
events_to_plot = ['TTR']
events_to_plot_ais = ['TTR'] 

fig, ax = plt.subplots(1,2, figsize=(15,5), sharex=True, sharey=True)

threshold_model = ThresholdModel()
ax[0], both_actions1, cut_bins = Graphs.plot_ucurve_new(val_buffer.data, model='threshold', groupby_col="ID", adverse_events=events_to_plot, incl_hist=True, outcome_agg_method="last", axes=ax[0], num_actions=num_actions, use_abs=True, use_qcut=True)
ax[0], both_actions2, _ = Graphs.plot_ucurve_new(val_buffer.data, model, groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[0], use_abs=True, cut_bins=cut_bins)
# ax[0], _, _ = Graphs.plot_ucurve_new(val_buffer.data, model='naive', groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[0], use_abs=True, cut_bins=cut_bins)
# ax[0], _, _ = Graphs.plot_ucurve_new(val_buffer.data, model='random', groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[0], use_abs=True, num_actions=7, cut_bins=cut_bins)


# ax[0].set_ylim([0,0.15])
ax[0].legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])
ax[0].set_ylabel('TTR')
ax[0].set_xlabel('Absolute Difference between Model and Clinician (% Dose Change)')
ax[0].set_title(f"Validation Data: \n{events_to_plot}")

Graphs.get_ci(both_actions1, 2);
Graphs.get_ci(both_actions2, 2);
print(f"percent of patients: {both_actions1[both_actions1['DIFF_ACTIONS_BIN'] == 0].shape[0] / both_actions1.shape[0]:,.2%}, num: {sum(both_actions1['DIFF_ACTIONS_BIN'] == 0)}")
print(f"percent of patients: {both_actions2[both_actions2['DIFF_ACTIONS_BIN'] == 0].shape[0] / both_actions2.shape[0]:,.2%}, num: {sum(both_actions2['DIFF_ACTIONS_BIN'] == 0)}")

both_actions11, both_actions12 = both_actions1, both_actions2

ax[1], both_actions1, cut_bins = Graphs.plot_ucurve_new(train_buffer.data, model='threshold', groupby_col="ID", adverse_events=events_to_plot, incl_hist=True, outcome_agg_method="last", axes=ax[1], num_actions=num_actions, use_abs=True, use_qcut=True)
ax[1], both_actions2, _ = Graphs.plot_ucurve_new(train_buffer.data, model, groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[1], use_abs=True, cut_bins=cut_bins)
# ax[1], _, _ = Graphs.plot_ucurve_new(train_buffer.data, model='naive', groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[1], use_abs=True, cut_bins=cut_bins)
# ax[1], _, _ = Graphs.plot_ucurve_new(train_buffer.data, model='random', groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[1], use_abs=True, num_actions=7, cut_bins=cut_bins)


ax[1].legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])
ax[1].set_xlabel('Absolute Difference between Model and Clinician (% Dose Change)')
ax[1].set_ylabel('TTR')
ax[1].set_title(f"Training Data: \n{events_to_plot}")

Graphs.get_ci(both_actions1, 2);
Graphs.get_ci(both_actions2, 2);
print(f"percent of patients: {both_actions1[both_actions1['DIFF_ACTIONS_BIN'] == 0].shape[0] / both_actions1.shape[0]:,.2%}, num: {sum(both_actions1['DIFF_ACTIONS_BIN'] == 0)}")
print(f"percent of patients: {both_actions2[both_actions2['DIFF_ACTIONS_BIN'] == 0].shape[0] / both_actions2.shape[0]:,.2%}, num: {sum(both_actions2['DIFF_ACTIONS_BIN'] == 0)}")

ax[0].set_ylim([0, 1])
# ax[0].set_title("")
plt.show()

fig, ax = plt.subplots(1,2, figsize=(15,5), sharex=True)
both_actions11['DIFF_ACTIONS_BIN'].hist(alpha=0.5, ax=ax[0])
both_actions12['DIFF_ACTIONS_BIN'].hist(alpha=0.5, ax=ax[0])
both_actions1['DIFF_ACTIONS_BIN'].hist(alpha=0.5, ax=ax[1])
both_actions2['DIFF_ACTIONS_BIN'].hist(alpha=0.5, ax=ax[1])
ax[0].set_title("Validation Data")
ax[1].set_title("Training Data")
ax[0].legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])
ax[1].legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])

### Investigate agreement between RELY and RL

In [ ]:
events_to_plot = ['TTR']

buffer_data = val_buffer.data

fig, ax = plt.subplots(1,2, figsize=(15,5), sharex=True, sharey=True)

threshold_model = ThresholdModel()
ax[0], both_actions1, cut_bins = Graphs.plot_ucurve_new(buffer_data, model='threshold', groupby_col="ID", adverse_events=events_to_plot, incl_hist=True, outcome_agg_method="last", axes=ax[0], num_actions=num_actions, use_abs=True, use_qcut=True)
ax[0], both_actions2, _ = Graphs.plot_ucurve_new(buffer_data, model, groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax[0], use_abs=True, cut_bins=cut_bins)

plt.close()
# ax[0].legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])
# ax[0].set_ylabel('TTR')
# ax[0].set_xlabel('Absolute Difference between Model and Clinician (% Dose Change)')
# ax[0].set_title(f"Validation Data: \n{events_to_plot}")

In [ ]:
both_actions1['Model'] = "RE-LY"
both_actions2['Model'] = 'RL'
temp_df = pd.concat([both_actions1, both_actions2], ignore_index=True)

In [ ]:
# sns.boxplot(data=temp_df, x="DIFF_ACTIONS_BIN", y="AGREE", hue='Model')
sns.violinplot(data=temp_df, x="DIFF_ACTIONS_BIN", y="ADV_EVENTS", hue='Model')

In [ ]:
print(f"RE-LY: -- {both_actions1[both_actions1['DIFF_ACTIONS_BIN'] == 0].shape[0]} entries at x=0")
print(f"RL: ----- {both_actions2[both_actions2['DIFF_ACTIONS_BIN'] == 0].shape[0]} entries at x=0")

In [ ]:
merged_actions = both_actions1.merge(both_actions2, left_index=True, right_index=True)
both_zero = merged_actions[(merged_actions['DIFF_ACTIONS_BIN_x'] == 0) & (merged_actions['DIFF_ACTIONS_BIN_y'] == 0)]
print(f"{both_zero.shape[0]} trajectories where both RE-LY and RL are at x=0")

In [ ]:
id_mapping = buffer_data.groupby('USUBJID_O_NEW')['SUBJID'].last()
subset_baseline = baseline[baseline['SUBJID'].isin(buffer_data['SUBJID'].unique())]

In [ ]:
both_zero = both_zero.merge(id_mapping, how="left", left_index=True, right_index=True)

In [ ]:
pat_ids = both_zero['SUBJID'].unique()
print(f"{len(pat_ids)} unique patients at x=0")
zero_patients = baseline[baseline['SUBJID'].isin(pat_ids)]

In [ ]:
only_rely = merged_actions[(merged_actions['DIFF_ACTIONS_BIN_x'] == 0) & (merged_actions['DIFF_ACTIONS_BIN_y'] != 0)]
only_rl = merged_actions[(merged_actions['DIFF_ACTIONS_BIN_y'] == 0) & (merged_actions['DIFF_ACTIONS_BIN_x'] != 0)]
print(f"{only_rely.shape[0]} trajectories in RE-LY x=0")
print(f"{only_rl.shape[0]} trajectories in RL x=0")

only_rely = only_rely.merge(id_mapping, how="left", left_index=True, right_index=True)
only_rl = only_rl.merge(id_mapping, how="left", left_index=True, right_index=True)

only_rely = baseline[baseline['SUBJID'].isin(only_rely['SUBJID'].unique())]
only_rl = baseline[baseline['SUBJID'].isin(only_rl['SUBJID'].unique())]

In [ ]:
print(f"{only_rely['SUBJID'].nunique()} unique patients in RE-LY x=0")
print(f"{only_rl['SUBJID'].nunique()} unique patients in RL x=0")

In [ ]:
only_rely['Patient Group'] = 'x=0 for RELY only'
only_rl['Patient Group'] = 'x=0 for RL only'

In [ ]:
zero_patients['Patient Group'] = "x=0 for both RELY and RL"

other_patients = baseline[~baseline['SUBJID'].isin(pat_ids)]
other_patients['Patient Group'] = 'Other Patients'

baseline['Patient Group'] = 'Entire Dataset'

all_patients = pd.concat([zero_patients, other_patients, only_rely, only_rl, baseline])

In [ ]:
# dev_data = pd.concat([subs0, subs1, subs2])
x, y, hue, title = "REGION", "% Dataset", "Patient Group", "Dist of Patient Demographic"

plt.figure(figsize=(10, 5))
Graphs.create_hist(all_patients, x, y, hue, title=title)
plt.xticks(rotation=25)

In [ ]:
x, y, hue, title = "SEX", "% Dataset", "Patient Group", "Dist of Patient Demographic"

plt.figure(figsize=(10, 5))
Graphs.create_hist(all_patients, x, y, hue, title=title)
plt.xticks(rotation=25)

In [ ]:
x, y, hue, title = "TRIAL", "% Dataset", "Patient Group", "Dist of Patient Demographic"

plt.figure(figsize=(10, 5))
Graphs.create_hist(all_patients, x, y, hue, title=title)
plt.xticks(rotation=25)

In [ ]:
x, y, hue, title = "HX_MI", "% Dataset", "Patient Group", "Dist of Patient Demographic"

plt.figure(figsize=(10, 5))
Graphs.create_hist(all_patients, x, y, hue, title=title)
plt.xticks(rotation=25)

In [ ]:
all_patients['AGE_DEIDENTIFIED'] = all_patients['AGE_DEIDENTIFIED'].apply(lambda x: 90 if x == ">89" else float(x))

In [ ]:
cut_bins = [-0.001, 50, 60, 65, 70, 75, 80, 91]
cut_labels = ["<=50", "(50, 60]", "(60, 65]", "(65, 70]", "(70, 75]", "(75, 80]", ">80"]

all_patients['AGE_BIN'] = pd.cut(all_patients['AGE_DEIDENTIFIED'], bins=cut_bins, labels=cut_labels)

counts = all_patients['AGE_BIN'].value_counts(normalize=True)
counts = counts.reindex(cut_labels)

plt.bar(x=counts.index, height=counts.values)
plt.xticks(rotation=45)
plt.xlabel("Age")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))

In [ ]:
x, y, hue, title = "AGE_BIN", "% Dataset", "Patient Group", "Dist of Patient Demographic"

plt.figure(figsize=(10, 5))
Graphs.create_hist(all_patients, x, y, hue, title=title)
plt.xticks(rotation=25)

### Performance across demographics

In [ ]:
colname = "CONTINENT_EAST ASIA"
Graphs.get_action_heatmap_by_feature(train_buffer, val_buffer, colname, num_actions, state_method, model)

In [ ]:
colname = "CONTINENT_NORTH AMERICA"
Graphs.get_action_heatmap_by_feature(train_buffer, val_buffer, colname, num_actions, state_method, model)

In [ ]:
colname = "CONTINENT_EASTERN EUROPE"
Graphs.get_action_heatmap_by_feature(train_buffer, val_buffer, colname, num_actions, state_method, model)

In [ ]:
val_buffer.data["CONTINENT"] = val_buffer.data[[x for x in val_buffer.data.columns if "CONTINENT" in x]].idxmax(axis=1)

In [ ]:
val_buffer.data["CONTINENT"].value_counts()

In [ ]:
sns.displot(val_buffer.data, x="TTR", hue="CONTINENT", kind="kde")

In [ ]:
events_to_plot = ['TTR']
buffer_data = deepcopy(val_buffer.data)
buffer_data = buffer_data.drop(columns=["SUBJID", "CONTINENT"])

colname1 = "CONTINENT_EAST ASIA"
colname2 = "CONTINENT_EASTERN EUROPE"

Graphs.get_ucurve_by_demographic_feature(buffer_data, model, events_to_plot, colname1, colname2)

In [ ]:
events_to_plot = ['TTR']
buffer_data = deepcopy(val_buffer.data)
buffer_data = buffer_data.drop(columns=["SUBJID", "CONTINENT"])

colname1 = "CONTINENT_EAST ASIA"
colname2 = "CONTINENT_NORTH AMERICA"

Graphs.get_ucurve_by_demographic_feature(buffer_data, model, events_to_plot, colname1, colname2)

In [ ]:
events_to_plot = ['TTR']
buffer_data = deepcopy(train_buffer.data)

colname1 = "CONTINENT_EAST ASIA"
colname2 = "CONTINENT_NORTH AMERICA"

Graphs.get_ucurve_by_demographic_feature(buffer_data, model, events_to_plot, colname1, colname2)

In [ ]:
events_to_plot = ['TTR']
buffer_data = deepcopy(train_buffer.data)

colname1 = "CONTINENT_EAST ASIA"
colname2 = "CONTINENT_EASTERN EUROPE"

Graphs.get_ucurve_by_demographic_feature(buffer_data, model, events_to_plot, colname1, colname2)

### Relationship between agreement and TTR

In [ ]:
events_to_plot = ['TTR']
events_to_plot_ais = ['TTR'] 

if "CONTINENT" in val_buffer.data:
    val_buffer.data = val_buffer.data.drop(columns="CONTINENT")

fig = plt.figure(figsize=(10,5))
ax = plt.gca()

threshold_model = ThresholdModel()
ax, both_actions1, cut_bins = Graphs.plot_ucurve_new(val_buffer.data, model='threshold', groupby_col="ID", adverse_events=events_to_plot, incl_hist=True, outcome_agg_method="last", axes=ax, num_actions=num_actions, use_abs=True, use_qcut=True)
ax, both_actions2, _ = Graphs.plot_ucurve_new(val_buffer.data, model, groupby_col="ID", adverse_events=events_to_plot_ais, incl_hist=True,  outcome_agg_method="last", axes=ax, use_abs=True, cut_bins=cut_bins)

ax.legend(['RE-LY Algorithm', 'RL Algorithm']) #, 'Do Nothing', 'Random'])
ax.set_ylabel('TTR')
ax.set_xlabel('Absolute Difference between Model and Clinician (% Dose Change)')
ax.set_title(f"Validation Data: \n{events_to_plot}");

In [ ]:
from sklearn.linear_model import LinearRegression
X = both_actions1['AGREE'].values.reshape(-1, 1)
y = both_actions1['ADV_EVENTS'].values
weight = both_actions1['TRAJ_LENGTH'].values

reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))


from sklearn.linear_model import LinearRegression
X = both_actions2['AGREE'].values.reshape(-1, 1)
y = both_actions2['ADV_EVENTS'].values

print()
reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

In [ ]:
both_actions = deepcopy(both_actions1)

both_actions['ID'] = both_actions.index.values
both_actions['SUBJID'] = both_actions['ID'].apply(lambda x: float(x.split(".")[0] + x.split(".")[-1])).values
both_actions = both_actions.merge(baseline[['SUBJID', 'REGION']], on ='SUBJID', how='left')
both_actions_agg = both_actions.groupby('REGION').mean()

ax = sns.regplot(data=both_actions_agg[both_actions_agg['AGREE'] <= 1], x="AGREE", y="ADV_EVENTS", ci=None)
ax.set_xlabel("% Agreement with Model")
ax.set_ylabel("TTR (%)")
ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.xaxis.set_major_formatter(PercentFormatter(1))
# ax.set_xlim([0.449, 0.474])
ax.set_xlim([0.43, 0.57])

from sklearn.linear_model import LinearRegression
X = both_actions_agg['AGREE'].values.reshape(-1, 1)
y = both_actions_agg['ADV_EVENTS'].values
weight = both_actions_agg['TRAJ_LENGTH'].values

reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

In [ ]:
both_actions = deepcopy(both_actions2)

both_actions['ID'] = both_actions.index.values
both_actions['SUBJID'] = both_actions['ID'].apply(lambda x: float(x.split(".")[0] + x.split(".")[-1])).values
both_actions = both_actions.merge(baseline[['SUBJID', 'REGION']], on ='SUBJID', how='left')
both_actions_agg = both_actions.groupby('REGION').mean()

ax = sns.regplot(data=both_actions_agg[both_actions_agg['AGREE'] <= 1], x="AGREE", y="ADV_EVENTS", ci=None)
ax.set_xlabel("% Agreement with Model")
ax.set_ylabel("TTR (%)")
ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.xaxis.set_major_formatter(PercentFormatter(1))
# ax.set_xlim([0.449, 0.474])
ax.set_xlim([0.43, 0.57])

from sklearn.linear_model import LinearRegression
X = both_actions_agg['AGREE'].values.reshape(-1, 1)
y = both_actions_agg['ADV_EVENTS'].values
weight = both_actions_agg['TRAJ_LENGTH'].values

reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

In [ ]:
ax = sns.regplot(data=both_actions1, x="AGREE", y="ADV_EVENTS")
ax.set_xlabel("% Agreement with Model")
ax.set_ylabel("TTR (%)")
ax.set_title("Threshold Model")
ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.xaxis.set_major_formatter(PercentFormatter(1))

X = both_actions11['AGREE'].values.reshape(-1, 1)
y = both_actions11['ADV_EVENTS'].values
weight = both_actions11['TRAJ_LENGTH'].values

reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

In [ ]:
ax = sns.regplot(data=both_actions2, x="AGREE", y="ADV_EVENTS")
ax.set_xlabel("% Agreement with Model")
ax.set_ylabel("TTR (%)")
ax.set_title("dBCQ Model")

ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.xaxis.set_major_formatter(PercentFormatter(1))


X = both_actions12['AGREE'].values.reshape(-1, 1)
y = both_actions12['ADV_EVENTS'].values
weight = both_actions12['TRAJ_LENGTH'].values

reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

reg = LinearRegression().fit(X,y,weight)
print(reg.coef_)
print(reg.score(X, y))
print(reg.predict(np.array([100, 50]).reshape(-1, 1)))

### Consistency

In [ ]:
def consistency(Q):
    """
    Measures whether a given action distribution is "consistent" ordinally.
    """
    D = np.diff(Q, axis=-1)

    # Impute leading zeros
    idxs = (D != 0.).argmax(axis=-1)
    for n, idx in enumerate(idxs):
        D[n, :idx] = 2.
    # Impute trailing zeros
    idxs = D.shape[1] - (np.flip(D, axis=-1) != 0.).argmax(axis=-1)
    for n, idx in enumerate(idxs):
        D[n, idx:] = -2.

    c = np.sign(-D)
    c_all = np.all(np.sort(c, axis=-1) == c, axis=-1)

    return c_all

In [ ]:
sample_state2 = deepcopy(val_buffer.state)
q, imt, _ = model.model(torch.FloatTensor(sample_state2).to("cpu"))
q

In [ ]:
imt = imt.exp()
imt = (imt / imt.max(1, keepdim=True)[0] > bcq_threshold).float()
# Use large negative number to mask actions from argmax
# return ((imt * q + (1. - imt) * -1e8).argmax(1)).astype(int)
# probs = pd.DataFrame([x for x in (imt * q + (1. - imt) * -1e8).to("cpu").detach().numpy()])
probs = pd.DataFrame(model.get_model_actions(sample_state2, return_prob=True, use_threshold=True))
probs = probs.replace(-1e8, np.nan)
Q = probs.ffill(axis=1).bfill(axis=1)
sum(consistency(Q))

In [ ]:
val_state_df = pd.DataFrame(val_buffer.state, columns=Constants.state_cols_mapping[state_method])
# val_state_df['consistency'] = consistency(Q)
X_full = val_state_df
y = consistency(Q)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, 20)
rfe = rfe.fit(X_full, y)
# print(rfe.support_)
# print(rfe.ranking_)

In [ ]:
state_cols = Constants.state_cols_mapping[state_method]

In [ ]:
X = X_full.loc[:, np.array(state_cols)[rfe.support_]]

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
# Largest positive drivers
pos_feats = pd.Series(result.params).nlargest(5)

indx = np.array(pos_feats.index)
for i in indx:
    print(f"{i}: \t {pos_feats[i]}")

In [ ]:
# Largest negative drivers 
neg_feats = pd.Series(result.params).nsmallest(5)

indx = np.array(neg_feats.index)
for i in indx:
    print(f"{i}: \t {neg_feats[i]}")

In [ ]:
probs = pd.DataFrame(q.detach().numpy())
diff_probs = pd.DataFrame(q.diff(axis=1).detach().numpy())

probs['mean'] = probs[[0,1,2,3,4,5,6]].mean(axis=1)
probs['action'] = probs.idxmax(axis=1)
probs['action_prob'] = probs.max(axis=1)
probs['dev_action'] = probs['action_prob'] - probs['mean']
probs['dev_action_pct'] = probs['dev_action'] / probs['mean']

plt.figure(figsize=(11,6))
sns.boxplot(data=probs, x='action', y='dev_action_pct')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.ylabel("% Deviation of Action Prob From Mean")
plt.xlabel("Action")

In [ ]:
imt = imt.exp()
imt = (imt / imt.max(1, keepdim=True)[0] > bcq_threshold).float()
# Use large negative number to mask actions from argmax
# return ((imt * q + (1. - imt) * -1e8).argmax(1)).astype(int)
# probs = pd.DataFrame([x for x in (imt * q + (1. - imt) * -1e8).to("cpu").detach().numpy()])
probs = pd.DataFrame(model.get_model_actions(sample_state2, return_prob=True, use_threshold=True))
probs = probs.replace(-1e8, np.nan)
probs['mean'] = probs[[0,1,2,3,4,5,6]].mean(axis=1)
probs['action'] = probs.idxmax(axis=1)
probs['action_prob'] = probs.max(axis=1)
probs['dev_action'] = probs['action_prob'] - probs['mean']
probs['dev_action_pct'] = probs['dev_action'] / probs['mean']

plt.figure(figsize=(11,6))
sns.boxplot(data=probs, x='action', y='dev_action_pct')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.ylabel("% Deviation of Action Prob From Mean")
plt.xlabel("Action")

In [ ]:
probs[(~probs[[0,1,2]].isnull().all(axis=1)) & (~probs[[4,5,6]].isnull().all(axis=1))]

In [ ]:
df_long = pd.melt(diff_probs, var_name="Index of Action", value_name="Difference")

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(data=df_long, x='Index of Action', y='Difference')

### Patient Clustering

Use PCA to reduce the dimensionality of the sate, and then use tSNE to vsualize the clusters. Investigate the actions in each cluster, and investigate how similar these clusters are

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [ ]:
sample_state = deepcopy(val_buffer.data)
sample_state = sample_state[~sample_state['WEIGHT'].isnull()]

sample_state.loc[:, "INR_1"] = sample_state.groupby('USUBJID_O_NEW')['INR_VALUE'].shift(1).fillna(
    sample_state["INR_VALUE"])
sample_state.loc[:, "INR_2"] = sample_state.groupby('USUBJID_O_NEW')['INR_VALUE'].shift(2).fillna(
    sample_state["INR_1"])
sample_state.loc[:, "INR_3"] = sample_state.groupby('USUBJID_O_NEW')['INR_VALUE'].shift(3).fillna(
    sample_state["INR_2"])
sample_state.loc[:, "INR_4"] = sample_state.groupby('USUBJID_O_NEW')['INR_VALUE'].shift(4).fillna(
    sample_state["INR_3"])

In [ ]:
# X = deepcopy(val_buffer.state)
cols = sample_state[Constants.state_cols_mapping[state_method]]
cols = [x for x in cols if x not in ["INR_1", "INR_2", "INR_3", "INR_4"]]
X = sample_state[cols].values

In [ ]:
cont_cols = [x for x in sample_state.columns if "CONTINENT_" in x]
sample_state['CONTINENT'] = sample_state[cont_cols].idxmax(axis=1).apply(lambda x: x.split("_")[1])

In [ ]:
cont_cols = [x for x in sample_state.columns if "AGE_BIN_" in x]
sample_state['AGE_BIN'] = sample_state[cont_cols].idxmax(axis=1).apply(lambda x: x.split("_")[-1])

Use PCA directly and visualize clusters

In [ ]:
pca = PCA(n_components=3)

pca_result = pca.fit_transform(X)
df = pd.DataFrame({})
ind = Constants.state_cols_mapping[state_method].index('CONTINENT_EAST ASIA')
# df['y'] = X[:,ind]
df['y'] = sample_state['CONTINENT']

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", df['y'].nunique()),
    data=df,
    legend="full",
    alpha=0.3
)

In [ ]:
pca = PCA(n_components=3)

pca_result = pca.fit_transform(X)
df = pd.DataFrame({})
ind = Constants.state_cols_mapping[state_method].index('CONTINENT_EAST ASIA')
# df['y'] = X[:,ind]
df['y'] = sample_state['AGE_BIN']

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", df['y'].nunique()),
    data=df,
    legend="full",
    alpha=0.3
)

In [ ]:
pca = PCA(n_components=3)

pca_result = pca.fit_transform(X)
df = pd.DataFrame({})
df['y'] = sample_state['SEX']

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", df['y'].nunique()),
    data=df,
    legend="full",
    alpha=0.3
)

#### PCA and k-means clustering

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(pca_result)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
kmeans_pca = KMeans(n_clusters=4, random_state=42)
kmeans_pca.fit(pca_result)

In [ ]:
sample_state['Cluster'] = kmeans_pca.labels_

In [ ]:
pca = PCA(n_components=3)

pca_result = pca.fit_transform(X)
df = pd.DataFrame({})
ind = Constants.state_cols_mapping[state_method].index('SEX')
# df['y'] = X[:,ind]
df['y'] = sample_state['Cluster']

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", df['y'].nunique()),
    data=df,
    legend="full",
    alpha=0.3
)

In [ ]:
pca = PCA(n_components=3)
sample_state['weight'] = np.where(sample_state[Constants.neg_reward_events].sum(axis=1) > 0, 10, 0)

pca_result = pca.fit_transform(X)
df = pd.DataFrame({})
df['y'] = sample_state['STROKE']
df['weight'] = sample_state['weight']

df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", df['y'].nunique()),
    data=df,
    legend="full",
    alpha=0.3
)

sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    color="",
    data=df[df['y']==1],
    legend="full",
    alpha=1
)

#### t-SNE 

In [ ]:
pca = PCA(n_components=10)
trans_X = pca.fit_transform(X)

In [ ]:
X_embedded = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300).fit_transform(trans_X)
X_embedded.shape

In [ ]:
df_subset = pd.DataFrame({})
ind = Constants.state_cols_mapping[state_method].index('SEX')
df_subset['y'] = sample_state['CONTINENT']
df_subset['tsne-2d-one'] = X_embedded[:,0]
df_subset['tsne-2d-two'] = X_embedded[:,1]

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", df_subset['y'].nunique()),
    data=df_subset,
    legend="full",
    alpha=0.3
)